## Fine tuning the convolutional layers

The code below shows how we can unfreeze last few layers to allow fine tuning

In [ ]:
from __future__ import print_function

import os
import json
import shutil
import numpy as np

from utils import prepare_data

from tensorflow.keras.models import Model, load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.applications import VGG19
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, roc_curve, \
                            precision_recall_curve, average_precision_score, confusion_matrix
import pickle
import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline

## Prepare Data

In [ ]:
data_path = "data"
models_path = "models"
valid_size = 0.2
FORCED_DATA_REWRITE = False

In [ ]:
train_path, valid_path = prepare_data(data_path=data_path, 
                                      valid_size=valid_size, 
                                      FORCED_DATA_REWRITE=FORCED_DATA_REWRITE)

In [ ]:
train_neg_path = os.path.join(train_path, "Negative")
train_pos_path = os.path.join(train_path, "Positive")
valid_neg_path = os.path.join(valid_path, "Negative")
valid_pos_path = os.path.join(valid_path, "Positive")

In [ ]:
img_height, img_width = 150, 150

In [ ]:
datagen = ImageDataGenerator(rescale=1. / 255)

In [ ]:
train_gen = datagen.flow_from_directory(train_path, 
                                        target_size=(img_height, img_width), 
                                        class_mode='binary', 
                                        batch_size=16, 
                                        shuffle=False)

valid_gen = datagen.flow_from_directory(valid_path, 
                                        target_size=(img_height, img_width), 
                                        class_mode='binary', 
                                        batch_size=16, 
                                        shuffle=False)

## Pre-trained Model as Feature Extractor

In [ ]:
conv_base = VGG19(include_top=False, 
                         weights="imagenet",  
                         input_shape=(img_height, img_width, 3))

conv_base.summary()

In [ ]:
model = Sequential()
model.add(conv_base)
model.add(GlobalAveragePooling2D())
model.add(Dense(units=512, activation="relu", kernel_initializer="he_normal"))
model.add(Dropout(rate=0.5))
model.add(Dense(units=1, activation="sigmoid"))

model.summary()

In [ ]:
conv_base.trainable = False
# only unfreeze the last conv block (block5_convx) 
for layer in conv_base.layers:
    if layer.name in ['block5_conv4']:
        layer.trainable = True


In [ ]:
model.summary()

In [ ]:
model.compile(loss="binary_crossentropy", 
                  optimizer=Adam(lr=0.0001), 
                  metrics=["accuracy"])

In [ ]:
train_steps_per_epoch = int(np.ceil(train_gen.n * 1. / train_gen.batch_size))
valid_steps_per_epoch = int(np.ceil(valid_gen.n * 1. / valid_gen.batch_size))

In [ ]:
history = model.fit(
      train_gen,
      steps_per_epoch=train_steps_per_epoch,
      epochs=20,
      validation_data=valid_gen, 
      validation_steps=valid_steps_per_epoch,
      verbose=2)

The validation accuracy seems to be worse than before (i.e. without fine-tuning the convolutional layer and just trained the classification dense layer). One reason maybe that our training samples are too little. 